In [16]:
from google.colab import drive
import matplotlib.pyplot as plt
from collections import OrderedDict
import statistics
import numpy as np
import pickle
import json

drive.mount('/content/drive');

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---

In [2]:
ROOT_PATH = "/content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto"
IMAGES_PATH = f"{ROOT_PATH}/new_images"

LABEL_INDEX = 3

In [3]:
MAX_SEQ_LENGTH = 30
NUM_FEATURES = 1024

In [4]:
users = [str(j).zfill(2) for j in range(1, 33)]
videos = list(range(40))

In [5]:
import pickle


def load_raw_data(path):
    with open(path, 'rb') as file:
        raw = pickle.load(file, encoding='latin-1')

    return raw["data"], raw["labels"]


def load_preprocessed_labels(user, threshold=None):
    file_path = f"{ROOT_PATH}/databases/DEAP/data_preprocessed_python/s{user}.dat"
    _, raw_labels = load_raw_data(file_path)

    likings = raw_labels[:, LABEL_INDEX]

    if threshold is None:
        threshold = np.mean(likings)
    labels = (likings > threshold) * 1

    return labels

In [6]:
from tensorflow import keras
import os

---

In [7]:
IMG_SIZE = 167


def build_feature_extractor():
    feature_extractor = keras.applications.DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")

In [8]:
NUM_USERS = 32
NUM_VIDEOS = 40
NUM_FRAMES = 30

In [9]:
feature_extractor = build_feature_extractor()

29097984/29084464 [==============================] - 0s 0us/step


In [20]:
import numpy as np
import cv2


NEW_FEATURES_PATH = f"{ROOT_PATH}/new_features"


for user in range(NUM_USERS):
    user_str = str(user + 1).zfill(2)

    new_user_features_path = os.path.join(NEW_FEATURES_PATH, f"usuario_{user_str}")

    try:
        os.mkdir(new_user_features_path)
    except FileExistsError:
        pass

    for video in range(NUM_VIDEOS):
        new_user_video_features_path = os.path.join(new_user_features_path, f"video_{video}")

        try:
            os.mkdir(new_user_video_features_path)
        except FileExistsError:
            pass

        print(f"User {user}, video {video}: ", end='')
        with open(f"{new_user_video_features_path}/features.npy", 'wb') as file:
            for idx, frame in enumerate(range(0, 8120, 280)):
                img_path = os.path.join(
                    f"{IMAGES_PATH}", 
                    f"usuario_{user_str}", 
                    f"video_{video}", 
                    f"fotograma_{frame}.png"
                )

                img = cv2.imread(img_path)[:, :, [2, 1, 0]]
                features = feature_extractor.predict(img[None, ...])

                np.save(file, features)
                print(idx, end=' ')
        print()

User 0, video 0: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 1: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 2: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 3: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 4: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 5: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 6: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 7: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 8: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 9: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
User 0, video 10: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 

---

In [ ]:
raise Exception  # no correr porque ya tenemos las labels correctas

for idx, user in enumerate([str(u).zfill(2) for u in range(1, 33)]):
    labels = load_preprocessed_labels(user)
    with open(os.path.join(ROOT_PATH, 'new_labels', f'user_{idx}.npy'), 'wb') as file:
        np.save(file, labels)